In [1]:
from config import gkey
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import gmaps

In [2]:
weather_df = pd.read_csv('Weather_Info.csv')
weather_df

,Unnamed: 0,Latitude,Longitude,Cities,Cloudiness,Humidity,Temperature,Wind Speed
0,0,-33.02,27.91,east london,0.0,93.0,64.40,6.93
1,1,65.61,-37.64,tasiilaq,92.0,53.0,24.80,18.34
2,2,-21.36,55.77,saint-philippe,0.0,62.0,76.33,3.36
3,3,-5.20,-35.46,touros,75.0,83.0,78.80,4.70
4,4,42.10,-79.24,jamestown,90.0,92.0,23.23,6.93
...,...,...,...,...,...,...,...,...
459,495,10.62,104.18,kampot,20.0,58.0,86.00,8.05
460,496,-21.42,-50.08,penapolis,0.0,57.0,75.20,23.04
461,497,-13.54,23.10,zambezi,100.0,95.0,66.81,6.62
462,498,1.57,30.25,bunia,79.0,59.0,64.94,4.03


In [3]:
gmaps.configure(api_key=gkey)
locations = weather_df[["Latitude","Longitude"]]
humidity = weather_df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
no_clouds = weather_df[weather_df['Cloudiness']==0]
light_breeze = no_clouds[no_clouds['Wind Speed']<10]
perfect_temp = light_breeze[light_breeze['Temperature']<80]
perfect_temp = perfect_temp[perfect_temp['Temperature']>70]
perfect_temp.reset_index(inplace=True)
perfect_temp.head(30)

,index,Unnamed: 0,Latitude,Longitude,Cities,Cloudiness,Humidity,Temperature,Wind Speed
0,2,2,-21.36,55.77,saint-philippe,0.0,62.0,76.33,3.36
1,51,53,10.95,-4.93,orodara,0.0,44.0,77.00,2.24
2,67,69,-33.65,115.33,busselton,0.0,46.0,75.00,5.01
3,210,230,10.15,-85.45,nicoya,0.0,94.0,75.20,2.24
4,228,248,17.02,54.09,salalah,0.0,64.0,71.60,3.36
5,390,424,15.49,-13.18,kanel,0.0,40.0,71.60,4.70
6,407,441,25.52,90.22,tura,0.0,38.0,73.18,4.50
7,428,463,-16.73,-43.86,montes claros,0.0,68.0,71.60,4.70
8,452,488,-21.34,55.48,saint-pierre,0.0,62.0,77.20,3.36
9,463,499,-9.67,-35.74,maceio,0.0,94.0,73.40,3.36


In [9]:
hotels = []
for i in range(len(perfect_temp)):
    target_coordinates = f"{perfect_temp['Latitude'][i]},{perfect_temp['Longitude'][i]}"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    place_data = response.json()
    if place_data['status'] != 'ZERO_RESULTS':
        lat = place_data["results"][0]['geometry']['location']["lat"]
        lng = place_data["results"][0]['geometry']['location']['lng']

        hotels.append({"Name":place_data['results'][0]['name'],
                                 "Location":(lat,lng),
                                 "City":place_data["results"][0]["plus_code"]['compound_code']})

hotels

[{'Name': 'Chambres d\'hôte "La Trinité"',
  'Location': (-21.358434, 55.77032209999999),
  'City': 'JQRC+J4 Saint-Philippe, Réunion'},
 {'Name': 'Bkf Orodara',
  'Location': (10.9778037, -4.903008),
  'City': 'X3HW+4Q Orodara, Burkina Faso'},
 {'Name': 'Observatory Guest House',
  'Location': (-33.64639289999999, 115.3486762),
  'City': '983X+CF Busselton WA, Australia'},
 {'Name': 'Guayacan Real',
  'Location': (10.1437256, -85.4324098),
  'City': '4HV9+F2 Guanacaste Province, Nicoya, Costa Rica'},
 {'Name': 'HAMDAN PLAZA HOTEL',
  'Location': (17.0193409, 54.0610108),
  'City': '2396+PC Salalah, Oman'},
 {'Name': 'Hotel RIKMAN Continental',
  'Location': (25.5126646, 90.21763349999999),
  'City': 'G679+33 Tura, Meghalaya, India'},
 {'Name': 'ibis Montes Claros Shopping',
  'Location': (-16.7404461, -43.8702787),
  'City': '745H+RV Cidade Nova, Montes Claros - State of Minas Gerais, Brazil'},
 {'Name': 'Lindsey Hôtel',
  'Location': (-21.341006, 55.47229069999999),
  'City': 'MF5C+HW

In [11]:
#lat = place_data["results"][0]['geometry']['location']['lat']
#lat.append(place_data["results"][0]['geometry']['location']['lat'])
#lng = place_data["results"][0]['geometry']['location']['lng']
#lng.append(place_data["results"][0]['geometry']['location']['lng'])
#marker_locations = []
#marker_locations.append((lat,lng))
#marker_locations
#lng
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
marker_locations = [marker['Location'] for marker in hotels]
hotel_info = [info_box_template.format(**hotel) for hotel in hotels]
marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig = gmaps.figure()
#markers = gmaps.marker_layer(marker_locations)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))